In [ ]:
import numpy as np
import pandas as pd
import re
from itertools import chain

In [ ]:
# (helps with displaying dataframes containing long strings)
pd.set_option('display.max_colwidth', 0)

# Load and clean ICD-9 <-> symptoms map

In [ ]:
temp_fp = "icd9_symptom_map_v2.csv"
icd9_symptoms_map = pd.read_csv(temp_fp)

In [ ]:
# Some ChatGPT responses included some extra preamble and formatting - remove this for ease of analysis
icd9_symptoms_map["symptoms_clean"] =  icd9_symptoms_map["symptoms"].apply(lambda x: x.split("Symptoms: ")[1])

In [ ]:
# Convert symptoms from string to list
icd9_symptoms_map["symptoms_list"] = icd9_symptoms_map["symptoms_clean"].apply(lambda x: [y.lower() for y in x.split(", ")])

In [ ]:
# Get full list of unique symptoms
unique_symptoms = list(set(chain.from_iterable(icd9_symptoms_map["symptoms_list"])))

# Extract symptoms from clinical notes

In [ ]:
notes = pd.read_csv("NOTEEVENTS.csv.gz")

In [ ]:
# For now, filter notes to only discharge summaries and only relevant columns
notes = notes.query("CATEGORY == 'Discharge summary'")
notes = notes[["HADM_ID", "TEXT"]]

In [ ]:
# Extract symptoms from each set of notes
temp_pattern = re.compile("|".join(map(re.escape, unique_symptoms)))
notes["symptoms"] = notes["TEXT"].apply(lambda x: temp_pattern.findall(x.lower()))

In [ ]:
# Save
notes.to_csv("notes_and_symptoms.csv", index=False)